# Izhikevich neuron | Uncertainty in modes

Simulate Izhikevich neuron for different modes from the original paper

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from sys import path as sys_path
from os.path import abspath as os_path_abspath
sys_path.append(os_path_abspath('..'))
import addpaths

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plot_utils as pltu

# Model parameters

In [ ]:
solver_method = 'FE'

In [ ]:
# Select modes to generate data for.
neuron_modes = [
    'Tonic spiking'              ,
    'Phasic spiking'             ,
    'Tonic bursting'             ,
    'Phasic bursting'            ,
    'Mixed mode'                 ,
    'Spike frequency adaptation' ,
    'Class 1'                    ,
    'Class 2'                    ,
    'Spike latency'              ,
    'Subthreshold oscillations'  ,
    'Resonator'                  ,
    'Integrator'                 ,
    'Rebound spike'              ,
    'Rebound burst'              ,
    'Threshold variability'      ,
    'DAP'                        ,
    'Inhibition-induced spiking' ,
    'Inhibition-induced bursting',
]

In [ ]:
import izhikevich_parameters
parameters = izhikevich_parameters.parameters()

# Generator

In [ ]:
import izhikevich
from data_generator_IN import data_generator_IN

gens = {}

for neuron_mode in neuron_modes:
    
    # Generate neuron instance.
    neuron_parameters, stimulus_parameters, t_parameters = parameters.select_mode(neuron_mode)
    neuron = izhikevich.neuron(neuron_parameters, stimulus_parameters)
    tmax = t_parameters['tmax']

    # Generate solver instance.
    gens[neuron_mode] = data_generator_IN(
        y0=neuron.y0, t0=0.0, h0=t_parameters['dt'], tmax=tmax,
        gen_acc_sols=False, return_vars=['ys', 'events'],
        model=neuron, n_samples=40, base_folder='_data/modes'
    )
    gens[neuron_mode].original_dt = t_parameters['dt']
    gens[neuron_mode].update_subfoldername(neuron=neuron_mode.replace('-', '_').replace(' ', '_'))

## Test

In [ ]:
gens['DAP'].gen_sol(method='FE', adaptive=0, plot=True, step_param=gens['DAP'].original_dt, pert_method='conrad')

In [ ]:
gens['DAP'].gen_sol(method='FE', adaptive=2, plot=True, step_param=gens['DAP'].original_dt, pert_method='conrad')

## Data

In [ ]:
for neuron_mode, gen in gens.items():
    gen.gen_and_save_data(
        method='FE', adaptive=0, pert_method='conrad', step_param=gen.original_dt,
        plot=True, overwrite=False,
    )

# Plot

In [ ]:
neuron_mode_plot_order = [
    'Phasic spiking'             ,
    'Tonic spiking'              ,
    'Tonic bursting'             ,
    'Phasic bursting'            ,
    'Mixed mode'                 ,
    'Spike frequency adaptation' ,
    'Class 1'                    ,
    'Class 2'                    ,
    'Spike latency'              ,
    'Subthreshold oscillations'  ,
    'Resonator'                  ,
    'Integrator'                 ,
    'Rebound spike'              ,
    'Rebound burst'              ,
    'Threshold variability'      ,
    'DAP'                        ,
    'Inhibition-induced spiking' ,
    'Inhibition-induced bursting',
]

In [ ]:
def plot_mode(ax, neuron_mode, adaptive, stim_y0=-105, stim_y1=-91, clip_on=True):
    """Plot traces and stimulus."""
    
    gen = gens[neuron_mode]
    
    data = gen.load_data_and_check(
        method='FE', step_param=gen.original_dt, adaptive=adaptive, pert_method='conrad')
    
    ts_org, vs_org, us_org, *_ = gen.model.original_solve(
        tmax=gen.tmax, dt=gen.original_dt, clip=False)

    pltu.plot_sample_trace(ax, ts_org, vs_org, label='org.', tr_kw=dict(clip_on=clip_on))
    
    pltu.plot_mean_and_uncertainty(
        ax, data.ts, data.vs, intpol_dt=0.1, smpidxs=np.random.choice(np.arange(gen.n_samples), 0),
        tr_mean_kw=dict(clip_on=clip_on), tr_bnds_kw=dict(clip_on=clip_on),
    )
        
    stim_ts = np.arange(gen.t0,gen.tmax,gen.original_dt/10)
    pltu.plot_stim_on_trace_plot(
        ax, ts=stim_ts, stim=[gen.model.get_I_at_t(t=t) for t in stim_ts],
        stim_y0=stim_y0, stim_y1=stim_y1
    )

In [ ]:
np.random.seed(42)

adaptive = 0
    
### Prepare axes ###
fig, axs = pltu.auto_subplots(
    len(neuron_mode_plot_order), xsize='text', max_nx_sb=3, sharey=True, ysizerow=0.9,
    subplot_kw=dict(ylim=(None, 40), yticks=([-50, 0]))
)

### Plot data ###
for ax, neuron_mode in zip(axs.flatten(), neuron_mode_plot_order):
    plot_mode(ax=ax, neuron_mode=neuron_mode, adaptive=adaptive, clip_on=True)
    ax.set_title(neuron_mode, loc='center', ha='center', fontsize=plt.rcParams['font.size'], va='top')

### Decorate ###
pltu.set_labs(axs, panel_nums='auto', panel_num_space=0, panel_num_va='top')

for ax in axs.flat: ax.spines['left'].set_bounds(-85, ax.get_ylim()[1])

sns.despine()
pltu.set_labs(axs[-1,:], xlabs='Time (ms)')
pltu.set_labs(axs[:,0], ylabs='v(t)')
pltu.tight_layout(h_pad=0.5, w_pad=0.3)

axs[0,0].legend(loc='upper right', frameon=False, bbox_to_anchor=(1,1.08), borderpad=0.1)

pltu.savefig(f"IN_modes{'_noovershoot' if adaptive==2 else ''}")
plt.show()
pltu.show_saved_figure(fig)